### Question 5 (10 marks)

In this question you will look at implied correlations and see how much moves in implied correlation contribute to moves in cross volatility, versus moves in the underlying USD-pair volatilities.

#### Solution

In [35]:
# python imports
import datetime

# 3rd party imports
import numpy
import pandas

def impliedCorr_analytics(pairx, pair1, pair2, tenor, sign, startDate, endDate):
    '''
    Function that construct two DataFrames: one holding day-to-day changes in the 
    cross ATM volatility, and one holding differences between the predicted cross 
    volatility (assuming the implied correlation from the day before) and the true 
    cross volatility. The function should print out statistics on both those series.

    
    :param pairx:     name of the cross pair (as 'AUDJPY') 
    :type  pairx:     string
    :param pair1:     name of the first pair (as 'AUDUSD') 
    :type  pair1:     string
    :param pair2:     name of the second pair (as 'USDJPY') 
    :type  pair2:     string
    :param tenor:     a string tenor (like '3m')
    :type  tenor:     string
    :param sign:      a flag to define whether the cross spot is the product or the ratio 
                      of the two USD spots (which affects the sign of the correlation)
    :type  sign:      bool, True if ratio, False if product
    :param startDate: the start date of the historical window
    :type  startDate: datetime
    :param endDate:   the end date of the historical window
    :type  endDate:   datetime
    
    '''
    
    # It should start by loading the data for the ATM implied volatility for the three 
    # tenors from the spreadsheet into pandas DataFrames 

    df = pandas.read_excel('fx_vol_data.xlsx')
    df = df[(df['Date']>=startDate) & (df['Date']<=endDate)]
    dfx = df[pairx+' '+tenor]
    df1 = df[pair1+' '+tenor]
    df2 = df[pair2+' '+tenor]

    # and then calculate a pandas DataFrame of implied correlations.

    impliedCorr = ((dfx**2 - df1**2 - df2**2) / (2.*df1*df2)) * (1-sign)
    
    # The next step: use the correlation from date i, along with the implied volatilities 
    # for the USD pairs on date i+1, to predict the cross volatility on date i+1. Do this 
    # with the pandas DataFrames you have already created.

    dfx_predicted = numpy.sqrt(df1**2 + df2**2 + 2*impliedCorr.shift(1)*df1*df2)
    
    # Finally, construct two DataFrames: one holding day-to-day changes in the cross ATM 
    # volatility, and one holding differences between the predicted cross volatility 
    # (assuming the implied correlation from the day before) and the true cross volatility.

    diffs_1d  = dfx - dfx.shift(1)
    diffs_err = dfx_predicted - dfx
    
    # The function should print out statistics on both those series.
    
    print('Statistics for day-to-day changes in the cross ATM vol:')
    print(diffs_1d.describe())
    print()
    
    print('Statistics for difference between the predicted and the true cross vol:')
    print(diffs_err.describe())
    print()

def main():    
    for tenor in ['1w','1m','6m','1y']:
        print('Tenor:',tenor)
        impliedCorr_analytics(
            pairx = 'AUDJPY',
            pair1 = 'AUDUSD',
            pair2 = 'USDJPY',
            tenor = tenor,
            sign = False,
            startDate = datetime.date(2007,1,1),
            endDate = datetime.date(2013,5,31),
        )
        print()

if __name__=='__main__':
    main()

Tenor: 1w
Statistics for day-to-day changes in the cross ATM vol:
count    1621.000000
mean        0.005345
std         1.795480
min       -15.880000
25%        -0.580000
50%         0.000000
75%         0.505000
max        18.805000
Name: AUDJPY 1w, dtype: float64

Statistics for difference between the predicted and the true cross vol:
count    1.621000e+03
mean     4.047207e-03
std      1.186038e+00
min     -1.392378e+01
25%     -3.995371e-01
50%     -1.776357e-15
75%      4.087137e-01
max      1.495141e+01
dtype: float64


Tenor: 1m
Statistics for day-to-day changes in the cross ATM vol:
count    1621.000000
mean        0.004625
std         1.096523
min        -8.627500
25%        -0.387500
50%        -0.020000
75%         0.302500
max        12.547500
Name: AUDJPY 1m, dtype: float64

Statistics for difference between the predicted and the true cross vol:
count    1621.000000
mean       -0.000849
std         0.742401
min        -7.311626
25%        -0.189330
50%        -0.002107
75%

Statistics for day-to-day changes in the cross ATM vol:

|      | 1w         | 1m         | 6m         | 1y         |
|------|------------|------------|------------|------------|
| mean | 0.005345   | 0.004625   | 0.003956   | 0.003694   |
| std  | 1.795480   | 1.096523   | 0.575049   | 0.431407   |
| min  | -15.880000 | -8.627500  | -3.722500  | -2.482500  |
| max  | 18.805000  | 12.547500  | 4.922500   |  3.717500  |

Statistics for difference between the predicted and the true cross vol:

|      | 1w         | 1m         | 6m         | 1y         |
|------|------------|------------|------------|------------|
| mean | 4.047207e-03  | -0.000849 | -0.000562  | -0.000429  |
| std  | 1.186038e+00  | 0.742401  | 0.383226   | 0.315963   |
| min  | -1.392378e+01 | -7.311626 | -3.415851  | -3.265282  |
| max  | 1.495141e+01  | 7.866179  | 4.922500   |  4.605420  |

**Conclusion:**    

** The hedge strategy works better for longer tenor since the implied correlation change within time in short term. **


Franklin Ma, Oct 4 2016